In [ ]:
#
# Replicates parts of "Hierarchical Edge Bundles: Visualization of Adjacency Relations in Hierarchical Data" by Danny Holten (2006)
#
import pandas as pd
import numpy  as np
from math import sin, cos, pi, sqrt
import random
import time
import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import *
rt = RACETrack()

In [ ]:
# Figure 3 alternate -- using the built in bezier curve for SVG (doesn't work for this technique)
pos = {}
pos['p0'], pos['p1'], pos['p2'], pos['p3'], pos['p4'] = (15,0), (5,3), (20,7), (35,3), (35,7)
pos['p10'], pos['p11'], pos['p12'], pos['p13'] = (-5,0), (5,7), (25,0), (45,0)
df = pd.DataFrame({'fm':['p0','p1','p2','p3',  'p1',  'p1',  'p3',   'p3'],
                   'to':['p1','p2','p3','p4',  'p10', 'p11', 'p12',  'p13']})
linknode = rt.linkNode(df,[('fm','to')], pos, link_arrow=False, w=640, h=256, bounds_percent=0.1)
svg      = linknode._repr_svg_()
svg      = svg[:svg.rindex('</svg>')]
svg += f'<path d="M {linknode.xT(pos["p0"][0])} {linknode.yT(pos["p0"][1])} ' + \
       f'C {linknode.xT(pos["p1"][0])} {linknode.yT(pos["p1"][1])} {linknode.xT(pos["p3"][0])} {linknode.yT(pos["p3"][1])} ' + \
       f'{linknode.xT(pos["p4"][0])} {linknode.yT(pos["p4"][1])}" stroke="#000000" fill="none"/>'
svg += '</svg>'
rt.displaySVG(svg)

In [ ]:
# Figure 3 - actual implementation from the paper
linknode = rt.linkNode(df,[('fm','to')], pos, link_arrow=False, w=640, h=256, bounds_percent=0.1)
svg      = linknode._repr_svg_()
svg      = svg[:svg.rindex('</svg>')]

# Formula 1 form the Holten Paper - generates a control point
def cP(i, n, beta, p_0, p_i, p_n_minus_1):
    _fn_ = lambda k: beta * p_i[k] + (1.0 - beta) * (p_0[k] + ((i/(n-1)) * (p_n_minus_1[k] - p_0[k])))
    return (_fn_(0),_fn_(1))

# Generate all the control points for the example
my_beta = 1.0
i, cps = 0, []
for _point_ in ['p0','p1','p2','p3','p4']:
    xy  = cP(i, 5, my_beta, pos['p0'], pos[_point_], pos['p4'])    
    xyt = (linknode.xT(xy[0]),linknode.yT(xy[1]))
    cps.append(xyt)
    svg += f'<line x1="{xyt[0]-5}" y1="{xyt[1]-5}" x2="{xyt[0]+5}" y2="{xyt[1]+5}" stroke="#000000" stroke-width="0.5" />'
    svg += f'<line x1="{xyt[0]+5}" y1="{xyt[1]-5}" x2="{xyt[0]-5}" y2="{xyt[1]+5}" stroke="#000000" stroke-width="0.5" />'
    i += 1

# For every four points, convert to a bezier curve and plot it...
for i in range(len(cps)-3):
    # Copied from wikipedia page on B-splines -- https://en.wikipedia.org/wiki/B-spline
    p0 = ( (1/6) * (cps[i][0] + 4*cps[i+1][0] + cps[i+2][0]) ,  (1/6) * (cps[i][1] + 4*cps[i+1][1] + cps[i+2][1]) )
    p1 = ( (1/3) * (2*cps[i+1][0] + cps[i+2][0]),               (1/3) * (2*cps[i+1][1] + cps[i+2][1]) )
    p2 = ( (1/3) * (cps[i+1][0] + 2*cps[i+2][0]),               (1/3) * (cps[i+1][1] + 2*cps[i+2][1]) )
    p3 = ( (1/6) * (cps[i+1][0] + 4*cps[i+2][0] + cps[i+3][0]), (1/6) * (cps[i+1][1] + 4*cps[i+2][1] + cps[i+3][1]) )
    _circle_ = lambda pt: f'<circle cx="{pt[0]}" cy="{pt[1]}" r="3" stroke="#ff0000" fill="none" />'
    svg += _circle_(p0) + _circle_(p1) + _circle_(p2) + _circle_(p3)
    svg += f'<path d="M {p0[0]} {p0[1]} C {p1[0]} {p1[1]} {p2[0]} {p2[1]} {p3[0]} {p3[1]}" stroke="#a0a0a0" stroke-width="8.0" fill="none"/>'

# For every four points, use the wikipedia interpolation... <which is the same as above... so.. issue is not here>
for i in range(len(cps)-3):
    # Copied from wikipedia page on B-splines -- https://en.wikipedia.org/wiki/B-spline
    b0,b1,b2,b3 = cps[i],cps[i+1],cps[i+2],cps[i+3]
    t, t_inc = 0.0, 0.05
    while t < 1.0:
        cT = lambda _t_, k: (1/6) * ( (-b0[k] + 3*b1[k] - 3*b2[k] +b3[k])*_t_**3 + (3*b0[k] - 6*b1[k] + 3*b2[k])*_t_**2 + (-3*b0[k] + 3*b2[k])*_t_ + (b0[k] + 4*b1[k] + b2[k]) )
        x1,y1 = cT(t,0),       cT(t,1)
        x2,y2 = cT(t+t_inc,0), cT(t+t_inc,1)
        svg += f'<line x1="{x1}" y1="{y1}" x2="{x2}" y2="{y2}" stroke="#000000" stroke-width="3" />'    
        t += t_inc

# Basis functions copied from https://math.stackexchange.com/questions/1964113/b-splines-of-degree-1-2-and-3
_b0_ = lambda _t_: (   _t_**2            )/2
_b1_ = lambda _t_: (-2*_t_**2 + 2*_t_ + 1)/2
_b2_ = lambda _t_: (   _t_**2 - 2*_t_ + 1)/2

# For the first three points, perform the interpolation...
t, t_inc = 0.5, 0.1
while t < 1.0:
    t0 = _b0_(t),       _b1_(t),       _b2_(t)
    t1 = _b0_(t+t_inc), _b1_(t+t_inc), _b2_(t+t_inc)
    x1,x2 = cps[0][0]*t0[0] + cps[1][0]*t0[1] + cps[2][0]*t0[2], cps[0][0]*t1[0] + cps[1][0]*t1[1] + cps[2][0]*t1[2]
    y1,y2 = cps[0][1]*t0[0] + cps[1][1]*t0[1] + cps[2][1]*t0[2], cps[0][1]*t1[0] + cps[1][1]*t1[1] + cps[2][1]*t1[2]
    svg += f'<line x1="{x1}" y1="{y1}" x2="{x2}" y2="{y2}" stroke="#00aa00" stroke-width="3" />'
    t += t_inc

# For the last three points, perform the interpolation...
t, t_inc = 0.0, 0.1
while t < 0.5:
    t0 = _b0_(t),       _b1_(t),       _b2_(t)
    t1 = _b0_(t+t_inc), _b1_(t+t_inc), _b2_(t+t_inc)
    x1,x2 = cps[-3][0]*t0[0] + cps[-2][0]*t0[1] + cps[-1][0]*t0[2], cps[-3][0]*t1[0] + cps[-2][0]*t1[1] + cps[-1][0]*t1[2]
    y1,y2 = cps[-3][1]*t0[0] + cps[-2][1]*t0[1] + cps[-1][1]*t0[2], cps[-3][1]*t1[0] + cps[-2][1]*t1[1] + cps[-1][1]*t1[2]
    svg += f'<line x1="{x1}" y1="{y1}" x2="{x2}" y2="{y2}" stroke="#00aa00" stroke-width="3" />'
    t += t_inc

# For the first two (and last two points), perform the interpolation
t, t_inc = 0.0, 0.1
while t < 0.4:
    # Basis function copied from https://math.stackexchange.com/questions/1964113/b-splines-of-degree-1-2-and-3
    _b0_ = lambda _t_: (1 - _t_)
    _b1_ = lambda _t_: (_t_)
    t0 = _b0_(t),       _b1_(t)
    t1 = _b0_(t+t_inc), _b1_(t+t_inc)
    x1,x2 = cps[0][0]*t0[0] + cps[1][0]*t0[1], cps[0][0]*t1[0] + cps[1][0]*t1[1]
    y1,y2 = cps[0][1]*t0[0] + cps[1][1]*t0[1], cps[0][1]*t1[0] + cps[1][1]*t1[1]
    svg += f'<line x1="{x1}" y1="{y1}" x2="{x2}" y2="{y2}" stroke="#0000ff" stroke-width="1.5" />'
    t0 = _b0_(t+0.5),       _b1_(t+0.5)
    t1 = _b0_(t+t_inc+0.5), _b1_(t+t_inc+0.5)
    x1,x2 = cps[-2][0]*t0[0] + cps[-1][0]*t0[1], cps[-2][0]*t1[0] + cps[-1][0]*t1[1]
    y1,y2 = cps[-2][1]*t0[0] + cps[-1][1]*t0[1], cps[-2][1]*t1[0] + cps[-1][1]*t1[1]
    svg += f'<line x1="{x1}" y1="{y1}" x2="{x2}" y2="{y2}" stroke="#0000ff" stroke-width="1.5" />'
    t += t_inc

svg += '</svg>'
rt.tile([svg])

In [ ]:
#
# piecewiseCubicBSpline() - interpolate piecewise cubic b-spline per "Hierarchical Edge Bundling" -- Holten (2006)
# - there's an error in the continuity between the 4th degree and 3rd degree connections (on both ends)
# -- see the hierarchical_edge_bundling.ipynb test file for an example
# - this version returns segments... which is suboptimal/inefficient...
#
def piecewiseCubicBSpline_segments(pts, beta=0.8):
    segments = []
    # Formula 1 form the Holten Paper - generates a control point
    def cP(i, n, p_0, p_i, p_n_minus_1):
        _fn_ = lambda k: beta * p_i[k] + (1.0 - beta) * (p_0[k] + ((i/(n-1)) * (p_n_minus_1[k] - p_0[k])))
        return (_fn_(0),_fn_(1))
    # Generate all the control points for the example
    i, cps = 0, []
    for i in range(len(pts)):
        xy = cP(i, len(pts), pts[0], pts[i], pts[-1])    
        cps.append(xy)

    # For the first two perform the interpolation
    t, t_inc = 0.0, 0.1
    while t < 0.4:
        # Basis function copied from https://math.stackexchange.com/questions/1964113/b-splines-of-degree-1-2-and-3
        _b0_ = lambda _t_: (1 - _t_)
        _b1_ = lambda _t_: (_t_)
        t0 = _b0_(t),       _b1_(t)
        t1 = _b0_(t+t_inc), _b1_(t+t_inc)
        x1,x2 = cps[0][0]*t0[0] + cps[1][0]*t0[1], cps[0][0]*t1[0] + cps[1][0]*t1[1]
        y1,y2 = cps[0][1]*t0[0] + cps[1][1]*t0[1], cps[0][1]*t1[0] + cps[1][1]*t1[1]
        segments.append(((x1,y1),(x2,y2)))
        t += t_inc

    # [1of2] Copy : Basis functions copied from https://math.stackexchange.com/questions/1964113/b-splines-of-degree-1-2-and-3
    _b0_ = lambda _t_: (   _t_**2            )/2
    _b1_ = lambda _t_: (-2*_t_**2 + 2*_t_ + 1)/2
    _b2_ = lambda _t_: (   _t_**2 - 2*_t_ + 1)/2

    # For the first three points, perform the interpolation...
    t, t_inc = 0.5, 0.1
    while t < 1.0:
        t0 = _b0_(t),       _b1_(t),       _b2_(t)
        t1 = _b0_(t+t_inc), _b1_(t+t_inc), _b2_(t+t_inc)
        x1,x2 = cps[0][0]*t0[0] + cps[1][0]*t0[1] + cps[2][0]*t0[2], cps[0][0]*t1[0] + cps[1][0]*t1[1] + cps[2][0]*t1[2]
        y1,y2 = cps[0][1]*t0[0] + cps[1][1]*t0[1] + cps[2][1]*t0[2], cps[0][1]*t1[0] + cps[1][1]*t1[1] + cps[2][1]*t1[2]
        segments.append(((x1,y1),(x2,y2)))
        t += t_inc

    # For every four points, use the wikipedia interpolation...
    # - it'd be faster to use the bezier implementation from SVG (see the test file) ... but if you want to colorize it,
    #   there's no implementation within SVG to shade across the curve...
    for i in range(len(cps)-3):
        # Copied from wikipedia page on B-splines -- https://en.wikipedia.org/wiki/B-spline
        b0,b1,b2,b3 = cps[i],cps[i+1],cps[i+2],cps[i+3]
        t, t_inc = 0.0, 0.05
        while t < 1.0:
            cT = lambda _t_, k: (1/6) * ( (-b0[k] + 3*b1[k] - 3*b2[k] +b3[k])*_t_**3 + (3*b0[k] - 6*b1[k] + 3*b2[k])*_t_**2 + (-3*b0[k] + 3*b2[k])*_t_ + (b0[k] + 4*b1[k] + b2[k]) )
            x1,y1 = cT(t,0),       cT(t,1)
            x2,y2 = cT(t+t_inc,0), cT(t+t_inc,1)
            segments.append(((x1,y1),(x2,y2)))
            t += t_inc

    # [2of2] Copy : Basis functions copied from https://math.stackexchange.com/questions/1964113/b-splines-of-degree-1-2-and-3
    _b0_ = lambda _t_: (   _t_**2            )/2
    _b1_ = lambda _t_: (-2*_t_**2 + 2*_t_ + 1)/2
    _b2_ = lambda _t_: (   _t_**2 - 2*_t_ + 1)/2

    # For the last three points, perform the interpolation...
    t, t_inc = 0.0, 0.1
    while t < 0.5:
        t0 = _b0_(t),       _b1_(t),       _b2_(t)
        t1 = _b0_(t+t_inc), _b1_(t+t_inc), _b2_(t+t_inc)
        x1,x2 = cps[-3][0]*t0[0] + cps[-2][0]*t0[1] + cps[-1][0]*t0[2], cps[-3][0]*t1[0] + cps[-2][0]*t1[1] + cps[-1][0]*t1[2]
        y1,y2 = cps[-3][1]*t0[0] + cps[-2][1]*t0[1] + cps[-1][1]*t0[2], cps[-3][1]*t1[0] + cps[-2][1]*t1[1] + cps[-1][1]*t1[2]
        segments.append(((x1,y1),(x2,y2)))
        t += t_inc

    # For the first two (and last two points), perform the interpolation
    t, t_inc = 0.0, 0.1
    while t < 0.4:
        # Basis function copied from https://math.stackexchange.com/questions/1964113/b-splines-of-degree-1-2-and-3
        _b0_ = lambda _t_: (1 - _t_)
        _b1_ = lambda _t_: (_t_)
        t0 = _b0_(t+0.5),       _b1_(t+0.5)
        t1 = _b0_(t+t_inc+0.5), _b1_(t+t_inc+0.5)
        x1,x2 = cps[-2][0]*t0[0] + cps[-1][0]*t0[1], cps[-2][0]*t1[0] + cps[-1][0]*t1[1]
        y1,y2 = cps[-2][1]*t0[0] + cps[-1][1]*t0[1], cps[-2][1]*t1[0] + cps[-1][1]*t1[1]
        segments.append(((x1,y1),(x2,y2)))
        t += t_inc
    return segments

In [ ]:
my_beta = 0.8
def plotSegments(segments,_color_='#000000'):
    svg = ''
    for s in segments:
        svg  += f'<line x1="{s[0][0]}" y1="{s[0][1]}" x2="{s[1][0]}" y2="{s[1][1]}" stroke="{_color_}" stroke-width="0.5" />'
    return svg
svg       = '<svg x="0" y="0" width="500" height="500" xmlns="http://www.w3.org/2000/svg">'
svg      += '<rect x="0" y="0" width="500" height="500" fill="#ffffff" />'
my_pts    = [(80,80),(100,120),(120,120),(160,200),(190,220),(230,220),(280,230),(330,200),(340,210),(345,230),(400,250)]
svg      += plotSegments(piecewiseCubicBSpline_segments(my_pts,my_beta))
my_pts    = [(80,60),(100,120),(120,120),(160,200),(190,220),(230,220),(280,230),(330,200),(340,210),(345,230),(370,250)]
svg      += plotSegments(piecewiseCubicBSpline_segments(my_pts,my_beta), '#00aa00')
my_pts    = [(80,40),(100,120),(120,120),(160,200),(190,220),(230,220),(280,230),(330,200),(340,210),(345,230),(350,260)]
svg      += plotSegments(piecewiseCubicBSpline_segments(my_pts,my_beta), '#0000aa')
my_pts    = [(50,200),(50,250),(25,200),(10,250),(10,400)]
svg      += plotSegments(piecewiseCubicBSpline_segments(my_pts,my_beta))
svg      += '</svg>'
rt.tile([svg])

In [ ]:
#
# piecewiseCubicBSpline() - interpolate piecewise cubic b-spline per "Hierarchical Edge Bundling" -- Holten (2006)
# - there's an error in the continuity between the 4th degree and 3rd degree connections (on both ends)
# -- see the hierarchical_edge_bundling.ipynb test file for an example
# - this version returns points which is more efficient for constructing svg structures
# - t_inc should be something that (when added to a single precision number (e.g., 0.1) 
#   will eventually add to another single precision number evenly)
#
def piecewiseCubicBSpline(pts, beta=0.8, t_inc=0.1):
    points = []
    # Formula 1 form the Holten Paper - generates a control point
    def cP(i, n, p_0, p_i, p_n_minus_1):
        _fn_ = lambda k: beta * p_i[k] + (1.0 - beta) * (p_0[k] + ((i/(n-1)) * (p_n_minus_1[k] - p_0[k])))
        return (_fn_(0),_fn_(1))
    # Generate all the control points for the example
    i, cps = 0, []
    for i in range(len(pts)):
        xy = cP(i, len(pts), pts[0], pts[i], pts[-1])    
        cps.append(xy)

    # For the first two perform the interpolation
    t = 0.0
    while t <= 0.4:
        # Basis function copied from https://math.stackexchange.com/questions/1964113/b-splines-of-degree-1-2-and-3
        _b0_ = lambda _t_: (1 - _t_)
        _b1_ = lambda _t_: (_t_)
        t0 = _b0_(t),       _b1_(t)
        x1,y1 = cps[0][0]*t0[0] + cps[1][0]*t0[1], cps[0][1]*t0[0] + cps[1][1]*t0[1]
        points.append((x1,y1))
        t += t_inc

    # [1of2] Copy : Basis functions copied from https://math.stackexchange.com/questions/1964113/b-splines-of-degree-1-2-and-3
    _b0_ = lambda _t_: (   _t_**2            )/2
    _b1_ = lambda _t_: (-2*_t_**2 + 2*_t_ + 1)/2
    _b2_ = lambda _t_: (   _t_**2 - 2*_t_ + 1)/2

    # For the first three points, perform the interpolation...
    t = 0.0
    while t <= 0.5:
        t0 = _b0_(t), _b1_(t), _b2_(t)
        x1,y1 = cps[2][0]*t0[0] + cps[1][0]*t0[1] + cps[0][0]*t0[2], cps[2][1]*t0[0] + cps[1][1]*t0[1] + cps[0][1]*t0[2] 
        points.append((x1,y1))
        t += t_inc

    # For every four points, use the wikipedia interpolation...
    # - it'd be faster to use the bezier implementation from SVG (see the test file) ... but if you want to colorize it,
    #   there's no implementation within SVG to shade across the curve...
    cT = lambda _t_, k: (1/6) * ( (-b0[k] + 3*b1[k] - 3*b2[k] +b3[k])*_t_**3 + (3*b0[k] - 6*b1[k] + 3*b2[k])*_t_**2 + (-3*b0[k] + 3*b2[k])*_t_ + (b0[k] + 4*b1[k] + b2[k]) )
    for i in range(len(cps)-3):
        # Copied from wikipedia page on B-splines -- https://en.wikipedia.org/wiki/B-spline
        b0,b1,b2,b3 = cps[i],cps[i+1],cps[i+2],cps[i+3]
        t = 0.0
        while t <= 1.0:
            x1,y1 = cT(t,0),       cT(t,1)
            points.append((x1,y1))
            t += t_inc

    # [2of2] Copy : Basis functions copied from https://math.stackexchange.com/questions/1964113/b-splines-of-degree-1-2-and-3
    _b0_ = lambda _t_: (   _t_**2            )/2
    _b1_ = lambda _t_: (-2*_t_**2 + 2*_t_ + 1)/2
    _b2_ = lambda _t_: (   _t_**2 - 2*_t_ + 1)/2

    # For the last three points, perform the interpolation...
    t = 0.5
    while t <= 1.0:
        t0 = _b0_(t), _b1_(t), _b2_(t)
        x1,y1 = cps[-1][0]*t0[0] + cps[-2][0]*t0[1] + cps[-3][0]*t0[2], cps[-1][1]*t0[0] + cps[-2][1]*t0[1] + cps[-3][1]*t0[2]
        points.append((x1,y1))
        t += t_inc

    # For the first two (and last two points), perform the interpolation
    t = 0.0
    while t <= 0.4:
        # Basis function copied from https://math.stackexchange.com/questions/1964113/b-splines-of-degree-1-2-and-3
        _b0_ = lambda _t_: (1 - _t_)
        _b1_ = lambda _t_: (_t_)
        t0 = _b0_(t+0.5),       _b1_(t+0.5)
        x1,y1 = cps[-2][0]*t0[0] + cps[-1][0]*t0[1], cps[-2][1]*t0[0] + cps[-1][1]*t0[1], 
        points.append((x1,y1))
        t += t_inc
    return points

# Figure 3 alternate -- using the built in bezier curve for SVG (doesn't work for this technique)
pos = {}
pos['p0'], pos['p1'], pos['p2'], pos['p3'], pos['p4'] = (15,0), (5,3), (20,7), (35,3), (35,7)
pos['p10'], pos['p11'], pos['p12'], pos['p13'] = (-5,0), (5,7), (25,0), (45,0)
df = pd.DataFrame({'fm':['p0','p1','p2','p3',  'p1',  'p1',  'p3',   'p3'],
                   'to':['p1','p2','p3','p4',  'p10', 'p11', 'p12',  'p13']})
linknode = rt.linkNode(df,[('fm','to')], pos, link_arrow=False, w=640, h=256, bounds_percent=0.1)
svg      = linknode._repr_svg_()
svg      = svg[:svg.rindex('</svg>')]
wpts, spts = [(15,0), (5,3), (20,7), (35,3), (35,7)], []
for _pt_ in wpts:
    spts.append((linknode.xT(_pt_[0]), linknode.yT(_pt_[1])))
# ===
pw = rt.piecewiseCubicBSpline(spts, beta=0.9, t_inc=0.01)
d  = f'M {pw[0][0]:0.2f} {pw[0][1]:0.2f}'
for i in range(1,len(pw)):
    d += f' L {pw[i][0]:0.2f} {pw[i][1]:0.2f}'
svg += f'<path d="{d}" stroke="#c0c0c0" fill="none" stroke-width="8" />'
pw = piecewiseCubicBSpline(spts, beta=0.9, t_inc=0.01)
d  = f'M {pw[0][0]:0.2f} {pw[0][1]:0.2f}'
for i in range(1,len(pw)):
    d += f' L {pw[i][0]:0.2f} {pw[i][1]:0.2f}'
svg += f'<path d="{d}" stroke="#ff0000" fill="none" />'
# ===
svg += '</svg>'
rt.tile([svg])

In [ ]:
#
# Hack to make it all work... but not consistent with 1st or 2nd order cubic b-splines...
#

# Figure 3 - actual implementation from the paper
linknode = rt.linkNode(df,[('fm','to')], pos, link_arrow=False, w=640, h=256, bounds_percent=0.1)
svg      = linknode._repr_svg_()
svg      = svg[:svg.rindex('</svg>')]

# Formula 1 form the Holten Paper - generates a control point
def cP(i, n, beta, p_0, p_i, p_n_minus_1):
    _fn_ = lambda k: beta * p_i[k] + (1.0 - beta) * (p_0[k] + ((i/(n-1)) * (p_n_minus_1[k] - p_0[k])))
    return (_fn_(0),_fn_(1))

# Generate all the control points for the example
my_beta = 1.0
i, cps = 0, []
for _point_ in ['p0','p1','p2','p3','p4']:
    xy  = cP(i, 5, my_beta, pos['p0'], pos[_point_], pos['p4'])    
    xyt = (linknode.xT(xy[0]),linknode.yT(xy[1]))
    cps.append(xyt)
    svg += f'<line x1="{xyt[0]-5}" y1="{xyt[1]-5}" x2="{xyt[0]+5}" y2="{xyt[1]+5}" stroke="#000000" stroke-width="0.5" />'
    svg += f'<line x1="{xyt[0]+5}" y1="{xyt[1]-5}" x2="{xyt[0]-5}" y2="{xyt[1]+5}" stroke="#000000" stroke-width="0.5" />'
    i += 1

# For every four points, convert to a bezier curve and plot it...
for i in range(len(cps)-3):
    # Copied from wikipedia page on B-splines -- https://en.wikipedia.org/wiki/B-spline
    p0 = ( (1/6) * (cps[i][0] + 4*cps[i+1][0] + cps[i+2][0]) ,  (1/6) * (cps[i][1] + 4*cps[i+1][1] + cps[i+2][1]) )
    p1 = ( (1/3) * (2*cps[i+1][0] + cps[i+2][0]),               (1/3) * (2*cps[i+1][1] + cps[i+2][1]) )
    p2 = ( (1/3) * (cps[i+1][0] + 2*cps[i+2][0]),               (1/3) * (cps[i+1][1] + 2*cps[i+2][1]) )
    p3 = ( (1/6) * (cps[i+1][0] + 4*cps[i+2][0] + cps[i+3][0]), (1/6) * (cps[i+1][1] + 4*cps[i+2][1] + cps[i+3][1]) )
    _circle_ = lambda pt: f'<circle cx="{pt[0]}" cy="{pt[1]}" r="3" stroke="#ff0000" fill="none" />'
    svg += _circle_(p0) + _circle_(p1) + _circle_(p2) + _circle_(p3)
    svg += f'<path d="M {p0[0]} {p0[1]} C {p1[0]} {p1[1]} {p2[0]} {p2[1]} {p3[0]} {p3[1]}" stroke="#000000" stroke-width="2.0" fill="none"/>'

# First Three Points ... hacking it into a bezier curve (not the same as the paper's recommendation for 2nd or 3rd order cubic b-splines)
pt_end   = ( (1/6) * (  cps[0][0] + 4*cps[1][0] + cps[2][0]) ,  (1/6) * (  cps[0][1] + 4*cps[1][1] + cps[2][1]) )
pt_end_d = ( (1/3) * (2*cps[1][0] +   cps[2][0]),               (1/3) * (2*cps[1][1] +   cps[2][1]) )
pt_mid_e = (pt_end[0] + (pt_end[0] - pt_end_d[0]), pt_end[1] + (pt_end[1] - pt_end_d[1]))
pt_beg   = linknode.xT(pos['p0'][0]), linknode.yT(pos['p0'][1]) 
pt_mid_b = (pt_beg[0] + pt_mid_e[0])/2, (pt_beg[1] + pt_mid_e[1])/2

d  = f'M {pt_beg[0]} {pt_beg[1]} '
d += f'C {pt_mid_b[0]} {pt_mid_b[1]} {pt_mid_e[0]} {pt_mid_e[1]} {pt_end[0]} {pt_end[1]}'
svg += f'<path d="{d}" fill="none" stroke="#ff0000"/>'

# Last Three Points .. hacking it into a bezier curve (not the same as the paper's recommendation for 2nd or 3rd order cubic b-splines)
pt_beg   = ( (1/6) * (cps[-3][0] + 4*cps[-2][0] + cps[-1][0]), (1/6) * (cps[-3][1] + 4*cps[-2][1] + cps[-1][1]) )
pt_beg_d = ( (1/3) * (cps[-3][0] + 2*cps[-2][0]),              (1/3) * (cps[-3][1] + 2*cps[-2][1]) )
pt_mid_b = pt_beg[0] + (pt_beg[0] - pt_beg_d[0]), pt_beg[1] + (pt_beg[1] - pt_beg_d[1])
pt_end   = linknode.xT(pos['p4'][0]), linknode.yT(pos['p4'][1])
pt_mid_e = (pt_end[0] + pt_mid_b[0])/2, (pt_end[1] + pt_mid_b[1])/2

d  = f'M {pt_beg[0]} {pt_beg[1]} '
d += f'C {pt_mid_b[0]} {pt_mid_b[1]} {pt_mid_e[0]} {pt_mid_e[1]} {pt_end[0]} {pt_end[1]}'
svg += f'<path d="{d}" fill="none" stroke="#ff0000"/>'

fn_x = lambda pt, _color_: f'<path d="M {pt[0]-5} {pt[1]-5} l 10 10 M {pt[0]+5} {pt[1]-5} l -10 10" fill="none" stroke="{_color_}"/>' 

svg += '</svg>'
rt.tile([svg])

In [ ]:
# Verify that the two methods replicated into framework are the same...
# Figure 3 - actual implementation from the paper
linknode = rt.linkNode(df,[('fm','to')], pos, link_arrow=False, w=640, h=256, bounds_percent=0.1)
svg      = linknode._repr_svg_() # if you don't do this first, the xT and yT functions aren't filled in...
svg      = svg[:svg.rindex('</svg>')]

world_coords = []
for _point_ in ['p0','p1','p2','p3','p4']:
    world_coords.append((linknode.xT(pos[_point_][0]), linknode.yT(pos[_point_][1])))

my_beta = 0.9
svg += f'<path d="{rt.svgPathCubicBSpline(world_coords, beta=my_beta)}" stroke="#000000" fill="none" stroke-width="6.0" />'

pts = rt.piecewiseCubicBSpline(world_coords, beta=my_beta)
for i in range(len(pts)-1):
    svg += f'<line x1="{pts[i][0]}" y1="{pts[i][1]}" x2="{pts[i+1][0]}" y2="{pts[i+1][1]}" stroke="#ff0000" stroke-width="2.0" />'

svg += '</svg>'
rt.tile([svg])

In [ ]:
# Verify that the ordering replicated into the framework is correct...
# Figure 3 - actual implementation from the paper
linknode = rt.linkNode(df,[('fm','to')], pos, link_arrow=False, w=640, h=256, bounds_percent=0.1)
svg      = linknode._repr_svg_() # if you don't do this first, the xT and yT functions aren't filled in...
svg      = svg[:svg.rindex('</svg>')]

world_coords = []
for _point_ in ['p0','p1','p2','p3','p4']:
    world_coords.append((linknode.xT(pos[_point_][0]), linknode.yT(pos[_point_][1])))

my_beta = 0.9

pts = rt.piecewiseCubicBSpline(world_coords, beta=my_beta)
for i in range(len(pts)-1):
    _color_ = rt.co_mgr.spectrum(i, 0, len(pts))
    svg += f'<line x1="{pts[i][0]}" y1="{pts[i][1]}" x2="{pts[i+1][0]}" y2="{pts[i+1][1]}" stroke="{_color_}" stroke-width="2.0" />'

svg += '</svg>'
rt.tile([svg])